In [ ]:
from DatabaseConnections import *
import params
import pandas as pd
import cPickle as cpickle
import itertools
import math
import os
import csv
import progressbar
from operator import itemgetter
from IPython.display import clear_output
%matplotlib inline
import matplotlib.pyplot as plt
import folium
# clear_output()

In [ ]:
db = PostgresDb(params.DB_HOST, params.DB_NAME) 

clear_output()
print 'Connected to database {}'.format(params.DB_NAME)

### Cleanup any files from previous run
#### -- This will delete old data --

In [ ]:
@timeDec
def reset(db, version):
    tables = ['node_stnameft', 'lion', 'node', 'c_master_node',
              'c_master_segs', 'C_Lion_%s' % version, 'C_Lion_Nodes_%s' % version, 'm']
    for t in tables:
        print 'Droping %s\n' % t
        db.query("DROP TABLE if exists %s" % t)
    return raw_input('Re-imported lion and nodes? <enter>\n').upper()

@timeDec
def reset_current_version(db, version):
    tables = ['node_stnameft', 'c_master_node',
              'c_master_segs', 'C_Lion_%s' % version, 'C_Lion_Nodes_%s' % version, 'm']
    for t in tables:
        print 'Droping %s\n' % t
        db.query("DROP TABLE if exists %s" % t)
    # clean up exsiting tables 
    db.query("alter table lion drop column version")
    db.query("alter table lion drop column exclude")
    db.query("alter table lion drop column mft")
    db.query("alter table lion drop column masteridfrom")
    db.query("alter table lion drop column masteridto")
    
    db.query("alter table node drop column version")
    db.query("alter table node drop column masterid")
    db.query("alter table node drop column is_int")
    print 'Done...'

In [ ]:
# clean out old versions of lion ?

if raw_input('Delete and reimport old LION (1) or clean up existing LION (2)') == '1':
    reset(db, params.VERSION)
else:
    reset_current_version(db, params.VERSION)

### Import Roadbed Pointer List into database

In [ ]:
def read_file(file_path):
    with open(file_path, 'r') as f:
        read_data = f.read()
    return read_data

def split_to_rows(raw_data):
    d = raw_data.split('\n')
    output = []
    for row in d:
        output.append(row.split(','))
    return output

def split_to_columns(data_rows):
    data = []
    # ['RPL_ID', 'SegmentIDG', 'SegmentIDR', 'RPC', 'NCI',
    #         'NodeLevelF', 'NodeLevelT', 'R_FrNd', 'G_FrNd', 'R_ToNd', 'G_ToNd']
    idn = 0
    for row in data_rows:
        if row != ['']:
            # print row
            idn += 1
            g_seg = row[0][:7]
            g_seg_typ = row[0][7]
            r_seg = row[0][8:15]
            rpc = row[0][16]
            nci = row[0][18]
            fnode_level_code = row[0][22]
            tnode_level_code = row[0][26]
            f_node_rb_seg = row[0][28:35]
            f_node_g_seg = row[0][36:43]
            t_node_rb_seg = row[0][44:51]
            t_node_g_seg = row[0][52:59]
            data.append([idn, g_seg, r_seg, rpc, nci, fnode_level_code, tnode_level_code,
                         f_node_rb_seg, f_node_g_seg, t_node_rb_seg, t_node_g_seg])
    return data

def add_to_db(db, rpl, data):
    # make sure table exists and is clean
    db.query("""DROP TABLE if exists {0};
                    CREATE TABLE {0}
                    (
                      rpl_id bigint, segmentidg bigint, segmentidr bigint,
                      rpc character varying(1), nci character varying(1),
                      nodelevelf character varying(1), nodelevelt character varying(1),
                      r_frnd bigint, g_frnd bigint, r_tond bigint, g_tond bigint
                    );
                """.format(rpl))
    for row in data:
        if row[0] != 'RPL_ID':
            add_row(db, row)


def add_row(db, row):
    if row[0]:
        db.query("""INSERT INTO tbl_rpl (rpl_id, segmentidg, segmentidr, rpc, nci,nodelevelf,
                        nodelevelt, r_frnd, g_frnd, r_tond, g_tond)
                        VALUES (%s);
                    """ % str(row)[1:-1])
        print 'Added %s' % row[0]
        clear_output()

In [ ]:
@timeDec
def add_rpl(db):
    r_data = read_file(os.path.join(params.FOLDER, params.RPL_TXT))
    dta = split_to_rows(r_data)
    split_data = split_to_columns(dta)
    add_to_db(db, params.RPL, split_data)

add_rpl(db)
query_to_table(db, "select * from {} limit 10".format(params.RPL))

In [ ]:
@timeDec
def save_dictionaries(io='OUT'):
    """write out the data in dictionaries, may only be useful for testing..."""
    if io == 'OUT':
        cpickle.dump(params.nodeStreetNames, open(os.path.join(params.FOLDER, "nodeStreetNames.p"), "wb"))
        cpickle.dump(params.nodeIsIntersection, open(os.path.join(params.FOLDER, "nodeIsIntersection.p"), "wb"))
        cpickle.dump(params.nodeNextSteps, open(os.path.join(params.FOLDER, "nodeNextSteps.p"), "wb"))
        cpickle.dump(params.segmentBlocks, open(os.path.join(params.FOLDER, "segmentBlocks.p"), "wb"))
        cpickle.dump(params.node_master, open(os.path.join(params.FOLDER, "nodeMaster.p"), "wb"))
        cpickle.dump(params.clusterIntersections, open(os.path.join(params.FOLDER, "clusterIntersections.p"), "wb"))
        cpickle.dump(params.streetSet, open(os.path.join(params.FOLDER, "streetSet.p"), "wb"))
        cpickle.dump(params.mft1Dict, open(os.path.join(params.FOLDER, "mft1Dict.p"), "wb"))
        cpickle.dump(params.altGraph, open(os.path.join(params.FOLDER, "altGraph.p"), "wb"))
        cpickle.dump(params.mfts, open(os.path.join(params.FOLDER, "mfts.p"), "wb"))
        cpickle.dump(params.coordFromMaster, open(os.path.join(params.FOLDER, "coordFromMaster.p"), "wb"))
    else:
        d1 = cpickle.load(open(os.path.join(params.FOLDER, "nodeStreetNames.p"), "rb"))
        d2 = cpickle.load(open(os.path.join(params.FOLDER, "nodeIsIntersection.p"), "rb"))
        d3 = cpickle.load(open(os.path.join(params.FOLDER, "nodeNextSteps.p"), "rb"))
        d4 = cpickle.load(open(os.path.join(params.FOLDER, "segmentBlocks.p"), "rb"))
        d5 = cpickle.load(open(os.path.join(params.FOLDER, "nodeMaster.p"), "rb"))
        d6 = cpickle.load(open(os.path.join(params.FOLDER, "clusterIntersections.p"), "rb"))
        d7 = cpickle.load(open(os.path.join(params.FOLDER, "streetSet.p"), "rb"))
        d8 = cpickle.load(open(os.path.join(params.FOLDER, "mft1Dict.p"), "rb"))
        d9 = cpickle.load(open(os.path.join(params.FOLDER, "altGraph.p"), "rb"))
        d10 = cpickle.load(open(os.path.join(params.FOLDER, "mfts.p"), "rb"))
        d11 = cpickle.load(open(os.path.join(params.FOLDER, "coordFromMaster.p"), "rb"))
        return d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11

# Preprocess 
## Set up data schema

In [ ]:
@timeDec
def pre_process(db, lion, node, version, highways=False):
    print 'Adding version number (%s)...\n' % version
    db.query("alter table %s add version varchar(50)" % lion)
    db.query("alter table %s add version varchar(50)" % node)
    db.query("update %s set version  = '%s'" % (lion, version))
    db.query("update %s set version  = '%s'" % (node, version))
    # add field for exclusion
    db.query("alter table %s add exclude int" % lion)
    db.query("update %s set exclude  = 1" % (lion)) # exclude everything
    
    print 'Adding master ID fields to tables...\n'
    db.query("alter table %s add column mft int, add column masteridfrom int, add column masteridto int" % lion)
    db.query("alter table %s add column masterid int" % node)
    db.query("alter table %s add column is_int int" % node)
    
    print 'Creating node street name table (highways = %s)...\n' % str(highways)
    if highways:
        db.query("""
                update {0} set exclude = 0  
                where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C') 
                and (nonped = 'D' or nonped is null)
                and trafdir != 'P' and rw_type!='7' -- take out ped streets (not including step streets)
                and street != 'UNNAMED STREET' 
                and street not like '%{1}%' and street not like '%{2}%'  and street not like '%{3}%'
                """.format(lion, 'PED OVPS', 'PEDESTRIAN OVERPASS', 'PEDESTRIAN UNDERPASS'))
        
        db.query("""
                update {0} set exclude = 0  
                where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C') 
                and (nonped = 'V' and street = 'PELHAM PARKWAY')
                and trafdir != 'P' and rw_type!='7' -- take out ped streets (not including step streets)
                and street != 'UNNAMED STREET' 
                and street not like '%{1}%' and street not like '%{2}%' and street not like '%{3}%'
                """.format(lion, 'PED OVPS', 'PEDESTRIAN OVERPASS', 'PEDESTRIAN UNDERPASS'))
        
        db.query("""
                update {0} set exclude = 0  
                where nonped = 'V' and rb_layer in ('B', 'G') and street = 'EAST FORDHAM ROAD'
                """.format(lion))
        
        db.query("""
                update {0} set exclude = 0  
                where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C') 
                and (nonped = 'V' and street = 'ROCKAWAY FREEWAY')
                and street != 'UNNAMED STREET' 
                and street not like '%{1}%' and street not like '%{2}%'  and street not like '%{3}%'
                """.format(lion, 'PED OVPS', 'PEDESTRIAN OVERPASS', 'PEDESTRIAN UNDERPASS'))
        
        db.query("""drop table if exists node_stnameFT;
                    create table node_stnameFT as (
                    select node::int, street, 0 as master from (
                        select nodeidfrom as node, street
                        from {0} where exclude = 0
                        group by nodeidfrom, street

                        union

                        select nodeidto as node, street
                        from {0} where exclude = 0
                        group by nodeidto, street
                    ) as included );""".format(lion))
    else:
        db.query("""update {0} set exclude = 0  
                where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C')
                and trafdir != 'P' and rw_type!='7' -- take out ped streets (not including step streets)
                and street != 'UNNAMED STREET' 
                and street not like '%{1}%' and street not like '%{2}%'  and street not like '%{3}%'
                """.format(lion, 'PED OVPS', 'PEDESTRIAN OVERPASS', 'PEDESTRIAN UNDERPASS'))
        
        db.query("""drop table if exists node_stnameFT;
                    create table node_stnameFT as (
                    select node::int, street, 0 as master
                    from (
                        select nodeidfrom as node, street
                        from {0}
                        where where exclude = 0
                        group by nodeidfrom, street

                        union

                        select nodeidto as node, street
                        from {0}
                        where where exclude = 0
                        group by nodeidto, street
                        ) ft
                    group by node, street)
                """.format(lion))
# Run preprocess
pre_process(db, params.LION, params.NODE, params.VERSION, params.HIGHWAYS)
# Check resutls


In [ ]:
query_to_table(db, "select * from node_stnameFT limit 10")

# Part 2
### Build data dictionaries

In [ ]:
def de_suffix(name):  # todo fix e as part of east example
    """cleans up street names for less percise matches"""
    d = {' WEST': ' DIR', ' EAST': ' DIR',
         ' SOUTH': ' DIR', ' NORTH': ' DIR',
         ' EXIT': ' DIR', ' ENTRANCE': ' DIR',
         ' APPROACH': ' DIR', ' NORTHBOUND': ' DIR',
         ' SOUTHBOUND': ' DIR', ' EASTBOUND': ' DIR',
         ' WESTBOUND': ' DIR'}  # new needs to be tested!!!!
    for i in d:
        if name not in ('WEST STREET', 'SOUTH STREET', 'NORTH STREET', 'EAST STREET',
                        'WEST AVENUE', 'SOUTH AVENUE', 'NORTH AVENUE', 'EAST AVENUE',
                        'WEST BOULEVARD', 'SOUTH BOULEVARD', 'NORTH BOULEVARD', 'EAST BOULEVARD',
                        'WEST LOOP', 'SOUTH LOOP', 'NORTH LOOP', 'EAST LOOP',
                        'WEST DRIVE', 'SOUTH DRIVE', 'NORTH DRIVE', 'EAST DRIVE',
                        'WEST ROAD', 'SOUTH ROAD', 'NORTH ROAD', 'EAST ROAD',
                        'JUNIPER BOULEVARD NORTH', 'JUNIPER BOULEVARD SOUTH'
                        'PROSPECT PARK WEST', 'AVENUE N', 'AVENUE S', 'AVENUE E',
                        'AVENUE W',):
            name = name.replace(i, d[i])
    return name 
    
@timeDec
def node_names(db, nodeStreetNames):
    data = db.query("select * from node_stnameFT")
    bar = progressbar.ProgressBar()
    for row in bar(data[0]):
        node, street = row[0], de_suffix(row[1])
        if node not in nodeStreetNames.keys():
            nodeStreetNames[node] = set([street])
        else:
            nodeStreetNames[node].add(street)
    return nodeStreetNames

params.nodeStreetNames = {}
params.nodeStreetNames = node_names(db, params.nodeStreetNames)

In [ ]:
@timeDec
def intersection_binary_temp(db, node_table, lion):
    """Updates the is_int field in db"""
    db.query("drop table if exists temp_int;")
    db.query("""create table temp_int as (
                    select nodeid, is_int, count(*) as cnt
                    from (
                        select nodeid, is_int, street
                        from {0} n join {1} l 
                        on n.nodeid::int = l.nodeidfrom::int 
                        where exclude = 0
                        union
                        select nodeid, is_int, street
                        from {0} n join {1} l 
                        on n.nodeid::int = l.nodeidto::int
                        where exclude = 0                        
                    ) as base
                    group by nodeid, is_int
                    having count(*) >1
                );""".format(node_table, lion))
    db.query("update {0} set is_int = -1 from temp_int where {0}.nodeid=temp_int.nodeid;".format(node_table))
    db.query("drop table temp_int;")  

intersection_binary_temp(db, params.NODE, params.LION)



In [ ]:
@timeDec
def intersection_highway_ramp_fix(db, node_table, lion):

    # create highway ramp intersection table
    db.query("""
               drop table if exists ramp_ints;
                create table ramp_ints  as 
                select nodeidfrom, nodelevelf, count(*) 
                from (
                    select nodeidfrom, nodelevelf, nonped from {0} 
                    where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') -- centerline vehicle only streets
                    and street != 'UNNAMED STREET' and street not like '%PED OVPS%' and street not like '%PEDESTRIAN OVERPASS%'
                    and exclude = 0
                    union select nodeidto, nodelevelt, nonped from {0} 
                    where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') -- centerline vehicle only streets
                    and street != 'UNNAMED STREET' and street not like '%PED OVPS%' and street not like '%PEDESTRIAN OVERPASS%'
                    and exclude = 0
                    
                    union
                     select nodeidfrom, nodelevelf, nonped from {0} 
                    where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') -- centerline vehicle only streets
                    and street != 'UNNAMED STREET' and street not like '%PED OVPS%' and street not like '%PEDESTRIAN OVERPASS%'
                    and (rw_type = '9' or segmenttyp = 'E') -- ramps
                    union select nodeidto, nodelevelt, nonped from {0} 
                    where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') -- centerline vehicle only streets
                    and street != 'UNNAMED STREET' and street not like '%PED OVPS%' and street not like '%PEDESTRIAN OVERPASS%'
                    and (rw_type = '9' or segmenttyp = 'E') -- ramps
                    
                    -- special case for error in LION 16D !!!!
                    union 
                    select nodeidfrom, nodelevelf, nonped from {0} 
                    where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') -- centerline vehicle only streets
                    and street in ('QUEENS MIDTOWN TUNNEL APPROACH', 'QUEENS MIDTOWN TUNNEL EXIT')
                    union select nodeidto, nodelevelt, nonped from {0} 
                    where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') -- centerline vehicle only streets
                    and street in ('QUEENS MIDTOWN TUNNEL APPROACH', 'QUEENS MIDTOWN TUNNEL EXIT')
                    -- special case for error in LION 16D !!!!
                    
                )i group by nodeidfrom, nodelevelf having count(*) > 1
    """.format(lion))
    # update main table
    db.query("update {0} set is_int = -1 from ramp_ints where {0}.nodeid::int=ramp_ints.nodeidfrom::int;".format(node_table))
    

intersection_highway_ramp_fix(db, params.NODE, params.LION)
    


In [ ]:
org,new = None, None
if 79419 in params.nodeStreetNames.keys():
    org = params.nodeStreetNames[79419]

In [ ]:
def intersection_highway_ramp_fix_part2(db, node_table, lion):
    # update node_stnameFT
    db.query("drop table if exists _temp_;")
    db.query("""create table _temp_ as 
                select n.nodeid, street, 0 as master
                from {0} l
                join (
                    select n.nodeid from {1} n
                    left outer join node_stnameFT s
                    on n.nodeid = s.node
                    where n.is_int = -1
                    and s.node is null
                ) as n on l.nodeidfrom::int = n.nodeid
                where exclude = 0
                union
                select n.nodeid, street, 0 as master
                from {0} l
                join (
                    select n.nodeid from {1} n
                    left outer join node_stnameFT s
                    on n.nodeid = s.node
                    where n.is_int = -1
                    and s.node is null
                ) as n on l.nodeidto::int = n.nodeid
                where exclude = 0
                """.format(lion, node_table))
    db.query("insert into node_stnameFT select * from _temp_;")
    db.query("drop table if exists _temp_;")
    # update node_stnameFT
#     db.query("""insert into node_stnameFT 
#                     select n.nodeid, street, 0 as master
#                     from {0} l
#                     join (
#                         select n.nodeid from {1} n
#                         left outer join node_stnameFT s
#                         on n.nodeid = s.node
#                         where n.is_int = -1
#                         and s.node is null
#                     ) as n on l.nodeidfrom::int = n.nodeid
#                     where exclude = 0
#                     union
#                     select n.nodeid, street, 0 as master
#                     from {0} l
#                     join (
#                         select n.nodeid from {1} n
#                         left outer join node_stnameFT s
#                         on n.nodeid = s.node
#                         where n.is_int = -1
#                         and s.node is null
#                     ) as n on l.nodeidto::int = n.nodeid
#                     where exclude = 0
#                     """.format(lion, node_table))
#     db.query("drop table ramp_ints;")
    # clean up

# run ramp intersectio patch
intersection_highway_ramp_fix_part2(db, params.NODE, params.LION)

### Update street name intersection data for ramps 

In [ ]:
def add_ramp_names_to_ints(db, lion):
    db.query("""
                insert into node_stnameFT
                select n.node, l.street, n.master
                from node_stnameFT n
                join ramp_ints r on n.node::int = r.nodeidfrom::int
                join (
                    select r.nodeidfrom, l.street
                    from ramp_ints r
                    join {0} l on r.nodeidfrom = l.nodeidfrom
                    union
                    select r.nodeidfrom, l.street
                    from ramp_ints r
                    join {0} l on r.nodeidfrom = l.nodeidto
                ) l on n.node::int = l.nodeidfrom::int and n.street != l.street
            """.format(lion))



add_ramp_names_to_ints(db, params.LION)
# regenerate streetnames with new ramp names
params.nodeStreetNames = node_names(db, params.nodeStreetNames)

print params.nodeStreetNames[79419]

In [ ]:
if 79419 in params.nodeStreetNames.keys():
    new = params.nodeStreetNames[79419]
print org, new, org == new 

In [ ]:
@timeDec
def get_double_blocks(db, lion, node_table):
    # get problem nodes where the blocks involved have more than 1 street name
    db.query("""drop table if exists doubles; 
                create table doubles as (
                    select nf::int, nt::int, count(street)
                    from (
                        select  nodeidfrom as nf, nodeidto as nt, street
                        from {0}
                        where exclude = 0 or (rw_type = '9' or segmenttyp = 'E') -- ramps
                        group by  nodeidfrom, nodeidto, street
                    union
                        select  nodeidto as nf, nodeidfrom as nt, street
                        from {0}
                        where exclude = 0 or (rw_type = '9' or segmenttyp = 'E') -- ramps
                        group by  nodeidfrom, nodeidto, street
                    ) as t
                    group by nf, nt
                    having count(street) > 1)""".format(lion))
    
    # then those nodes need to have > 2 streets to be an intersection. 
    db.query("""create table temp_int as (
                    select nodeid, is_int, count(*) as cnt from (
                            select nodeid, is_int, street
                            from (
                                    select nodeid, is_int, street
                                    from {0} 
                                    join {1} on st_dwithin({0}.geom, {1}.geom,1)
                                    where exclude = 0 or (rw_type = '9' or segmenttyp = 'E') -- ramps
                                    group by nodeid, is_int, street
                            ) as base, 
                    doubles
                    where nodeid = nf or nodeid = nt
                            group by nodeid, is_int, street
                    ) as grp group by nodeid, is_int
                            having count(*) < 3
                );""".format(node_table, lion))
    # update revised nodes
    db.query("""update %s as n
                    set is_int = Null
                    from temp_int
                    where n.nodeid =temp_int.nodeid
                    and n.is_int = -1
                    ;""" % node_table)
    db.query("drop table temp_int; drop table doubles;")


# fix locations with double segments...
get_double_blocks(db, params.LION, params.NODE)  # 149.80 sec



In [ ]:

def intersection_binary_dict(db, node_table, nodeIsIntersection):
    # get data from db into local dictionary
    data = db.query("select nodeid, is_int from %s " % node_table)  # where is_int =-1
    bar = progressbar.ProgressBar()
    for i in bar(data[0]):
        if i[1] == -1:
            nodeIsIntersection[int(i[0])] = True
        else:
            nodeIsIntersection[int(i[0])] = False
    del data
    print 'Dictionary updated'
    return nodeIsIntersection

params.nodeIsIntersection = intersection_binary_dict(db, params.NODE, params.nodeIsIntersection)
# # run this again for highway ramps
# params.nodeStreetNames = node_names(db, params.nodeStreetNames)

In [ ]:
# nodeStreetNames = {}
# # run this again for highway ramps
# nodeStreetNames = node_names(db, params.nodeStreetNames)
#print nodeStreetNames[43709]
#print params.nodeStreetNames[43709]

# print nodeStreetNames[46444] # 41 ave and bqe not intersection
# print params.nodeStreetNames[46444] # 41 ave and bqe not intersection
#print params.nodeIsIntersection[46444] # 41 ave and bqe not intersection


#print nodeStreetNames[46404] # northern and bqe exit
print params.nodeStreetNames[46404] # northern and bqe exit
print params.nodeIsIntersection[46404] # northern and bqe exit


print params.nodeIsIntersection[35485] # Rockaway Freeway and Beach 59 St
print params.nodeStreetNames[35485] # Rockaway Freeway and Beach 59 St

In [ ]:
def add_to_graph(street, node1, node2, nodeNextSteps):
    """checks if segment is in the graph adds it if not"""
    if node1 not in nodeNextSteps.keys():
        nodeNextSteps[node1] = {street: set([node2])}
    else:
        if street not in nodeNextSteps[node1].keys():
            nodeNextSteps[node1][street] = set([node2])
        else:
            nodeNextSteps[node1][street].add(node2)
    return nodeNextSteps


@timeDec
def graph(db, lion, nodeNextSteps, highways=False):
    """Graph the street network"""
    if highways:
        segs = db.query("""select street, segmentid, nodeidfrom, nodeidto 
                        from {0} where exclude = 0
                        """.format(lion))
    else:
        segs = db.query("""select street, segmentid, nodeidfrom, nodeidto 
                        from {0} where exclude = 0
                        """.format(lion))

    print 'Done with DB'
    bar = progressbar.ProgressBar()
    for row in bar(segs[0]):
        street, segmentid, nodeidfrom, nodeidto = row[0], row[1], int(row[2]), int(row[3])
        nodeNextSteps = add_to_graph(street, nodeidfrom, nodeidto, nodeNextSteps)
        nodeNextSteps = add_to_graph(street, nodeidto, nodeidfrom, nodeNextSteps)
    del segs
    return nodeNextSteps


params.nodeNextSteps = graph(db, params.LION, params.nodeNextSteps, params.HIGHWAYS)  # 179.22 sec

### Break point - save progress

In [ ]:
save_dictionaries('OUT')  # 19.72 sec
# params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 sec

# Part 3
### Build simple blocks

In [ ]:
# run to read saved data back into memory 

params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 se

In [ ]:
# simple graph search for grouping blocks

def go_to_end(street, done, todo, nodeIsIntersection=params.nodeIsIntersection, nodeNextSteps=params.nodeNextSteps):
    # anything left in the queue?
    if len(todo) == 0:
        # you're done exit
        return done
    else:
        # get new starting point
        start = todo.pop()
        # add it to the visited nodes
        done.append(start)
        # make sure it is not an ending point
        if not nodeIsIntersection[start]:
            # see where we can go from here
            for i in nodeNextSteps[start][street]:
                # make sure we haven't done it yet
                if i not in done:
                    # add it to the queue
                    todo.add(i)
        return go_to_end(street, done, todo)
    
@timeDec
def search(nodeStreetNames, streetSet, nodeIsIntersection, nodeNextSteps):
    """gets intersection to intersection segments or collection of segments"""
    for startNode in nodeStreetNames.keys():
        if nodeIsIntersection[startNode]:
            for street in nodeNextSteps[startNode].keys():
                queue = list(nodeNextSteps[startNode][street])
                while len(queue) > 0:
                    to_node = queue.pop()
                    streetSet.append(go_to_end(street, [startNode], set([to_node])))
    return nodeStreetNames, streetSet

params.nodeStreetNames, params.streetSet = search(params.nodeStreetNames, params.streetSet, params.nodeIsIntersection, params.nodeNextSteps)

print '\nnodeStreetNames(dict) sample: {{{0} : {1}}}'.format(params.nodeStreetNames.keys()[50], 
                                                             params.nodeStreetNames[params.nodeStreetNames.keys()[50]])

In [ ]:
nodeStreetNames, streetSet, nodeIsIntersection, nodeNextSteps = params.nodeStreetNames, params.streetSet, params.nodeIsIntersection, params.nodeNextSteps

for startNode in nodeStreetNames.keys():
        if nodeIsIntersection[startNode]:
            for street in nodeNextSteps[startNode].keys():
                queue = list(nodeNextSteps[startNode][street])
                while len(queue) > 0:
                    to_node = queue.pop()
                    streetSet.append(go_to_end(street, [startNode], set([to_node])))

                    
                    

In [ ]:
def write_csv(out_file, data_to_write, header=None, delim=','):
    row_cnt = 0
    with open(out_file, 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter=delim)
        if header:
            writer.writerow(header)
        for row in data_to_write:
            writer.writerow(row)  # this writes the rows to the csv file row needs to be a list
            row_cnt += 1
    return str(row_cnt) + " rows were written to " + str(out_file)

In [ ]:
print 'Starting...\n'
@timeDec
def mft_round1(db, lion, altGraph, streetSet, mft1Dict, folder, highways=False):
    """gets simplest master segments based on from to nodes, intersections only"""
    sql = """select segmentid, nodeidfrom, nodeidto
            from {0} where exclude = 0
            group by segmentid, nodeidfrom, nodeidto""".format(lion)

    data = db.query(sql)
    # build local dict of seg node ids from-to to build mft
    bar = progressbar.ProgressBar()
    for row in bar(data[0]):
        # build up simple graph
        # node : [[other node , segid], [other node , segid]]
        segment, fromid, toid = row
        if int(fromid) not in altGraph.keys():
            altGraph[int(fromid)] = [[int(toid), segment]]
        else:
            altGraph[int(fromid)].append([int(toid), segment])

        if int(toid) not in altGraph.keys():
            altGraph[int(toid)] = [[int(fromid), segment]]
        else:
            altGraph[int(toid)].append([int(fromid), segment])
    print 'done graphing\n'
    mft = 0
    to_write_out = []
    bar = progressbar.ProgressBar()
    for block in bar(streetSet):  # streetSet is a set of nodes that together make up a block
        if block:
            #print mft
            #clear_output()
            mft += 1
            for node in block:
                for pair in altGraph[node]:
                    if pair[0] in block:  # both ends of the street are in the block
                        if mft not in mft1Dict.keys():
                            mft1Dict[mft] = []
                        mft1Dict[mft].append(pair[1])
                        to_write_out.append([mft, pair[1]])
    print 'Writing out csv\n'
    write_csv(os.path.join(folder, "mft.csv"), to_write_out, ['mft', 'segment'])  
    db.query('drop table if exists tempMaster; CREATE TABLE tempMaster (mft varchar(10), seg varchar(10))')
    return altGraph

params.altGraph = mft_round1(db,  params.LION, params.altGraph,  params.streetSet, params.mft1Dict, 
                             params.FOLDER, params.HIGHWAYS)  # 1367.67 sec

print '\naltGraph(dict) sample: {{{0} : {1}}}'.format(params.altGraph.keys()[50], 
                                                             params.nodeStreetNames[params.altGraph.keys()[50]])

In [ ]:
import_table(db, 'tempMaster', os.path.join(params.FOLDER, 'mft.csv'))
# raw_input("""PG Console \n\n \copy tempMaster FROM '%s\mft.csv' DELIMITERS ',' CSV HEADER;\n\nDone?""" % params.FOLDER)
# clear_output()

In [ ]:
@timeDec
def add_temp_table(db, lion):  # Assumes that tempMaster is imported
    db.query("""update %s as l
                    set mft = t.mft::int
                    from tempmaster as t
                    where l.segmentid =t.seg""" % lion)
    db.query("drop table tempmaster")

add_temp_table(db, params.LION)  # 35.44 sec

In [ ]:
save_dictionaries('OUT')  # 10.07 sec

# Part 4
### Build simple intersections

In [ ]:
# run to read saved data back into memory 

params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 se

In [ ]:
@timeDec
def add_master_place_to_street_names(nodeStreetNames, nodeIsIntersection):
    to_remove = []
    for n in nodeStreetNames:
        if nodeIsIntersection[n]:  # only want intersection nodes
            # reformat the dict to have a spot for master id
            nodeStreetNames[n] = [nodeStreetNames[n], 0]
        else:
            # store non-intersections to remove latter
            to_remove.append(n)
    for i in to_remove:
        # not intersection remove it from dictionary
        del nodeStreetNames[i]
    return nodeStreetNames

params.nodeStreetNames = add_master_place_to_street_names(params.nodeStreetNames, params.nodeIsIntersection)

In [ ]:
def street_name_key(street_set):
    street_set = list(street_set)
    street_set.sort()
    street_set = str(street_set)
    return street_set[1:-1]

@timeDec
def intersection_cluster_dict(nodeStreetNames, clusterIntersections):
    # get street names by intersetion
    bar = progressbar.ProgressBar()
    for node in bar(nodeStreetNames.keys()):
        # order the street names
        street_key = street_name_key(nodeStreetNames[node][0])
        # if not in dictionary, add street name set
        if street_key not in clusterIntersections.keys():
            clusterIntersections[street_key] = [set(), 0]
        # add node to the street name set dict
        clusterIntersections[street_key][0].add(node)
    return clusterIntersections

params.clusterIntersections = intersection_cluster_dict(params.nodeStreetNames, params.clusterIntersections)

In [ ]:
@timeDec
def master_intersection_first_pass(clusterIntersections, node_master):
    master = 0
    for street_set in clusterIntersections.keys():
        master += 1
        # add masterid to each set of street names
        clusterIntersections[street_set][-1] = master
        # update node: master dict
        for node in clusterIntersections[street_set][0]:
            node_master[node] = master
    return clusterIntersections, node_master

params.clusterIntersections, params.node_master = master_intersection_first_pass(params.clusterIntersections, params.node_master)

In [ ]:
def distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [ ]:
# master = "'QUEENS PLAZA DIR'"
# #master = "'PEDESTRIAN OVERPASS'"
# node = 82294
# print params.clusterIntersections[master]
# print params.nodeStreetNames[node]

In [ ]:
@timeDec
def check_distance(db, node_table, folder, clusterIntersections):
    tolerance = 1000  # used to be 300.........
    data = db.query("select nodeid, st_x(geom), st_y(geom) from %s where is_int=-1" % node_table)
    loc = {}
    # get geometry
    for row in data[0]:
        node, x, y = row
        loc[node] = [x, y]
    # save the loc table for later
    cpickle.dump(loc, open(os.path.join(folder, "loc.p"), "wb"))

    problems = []
    bar = progressbar.ProgressBar()
    for master in bar(clusterIntersections.keys()):
        print master 
        
        # look for sets 2 nodes
        if len(clusterIntersections[master][0]) > 1:
            # get node's geom and check dist
            # n1, n2 = list(clusterIntersections[master][0])[:2]
            for pair in itertools.product(list(clusterIntersections[master][0]), repeat=2):
                n1, n2 = pair
                if distance(loc[n1][0], loc[n1][1], loc[n2][0], loc[n2][1]) > tolerance:
                    problems.append(master)
        clear_output()
    return problems

problems = check_distance(db, params.NODE, params.FOLDER, params.clusterIntersections)
# sample
query_to_table(db, "select nodeid, st_x(geom), st_y(geom) from %s where is_int=-1 limit 10" % params.NODE)

In [ ]:
# @timeDec
def update_problem_groups(problems, neighbors, node_master):
    for problem in problems:
        pcts = problem.keys()
        # simple case
        if len(pcts) == 2:
            if pcts[1] not in neighbors[pcts[0]]:  # pcts are not neighbors so update
                mx = get_max_master(node_master)
                for node in problem[pcts[1]]:
                    node_master[node] = mx + 1
                    # complex case with more than 2 pcts for now assume its a problem and update
                    # come back to this if becomes messy
        for pct in pcts:
            mx = get_max_master(node_master)
            for node in problem[pct]:
                node_master[node] = mx + 1
    return node_master

def get_max_master(node_master):
    # returns the largest master nodeid in the dict
    mx_master = 0
    for n in node_master.keys():
        if node_master[n] > mx_master:
            mx_master = node_master[n]
    return mx_master

@timeDec
def check_problem_pcts(db, problems, node_table, lion_table, precinct_table, clusterIntersections, node_master):
    # make lookup table
    db.query("drop table if exists c_intersection_name")
    # this fails if all the streets' connect to the node are "from" or if all the streets' connect to the node are "to"
    db.query("""create table c_intersection_name as (
                    select nodeid::int as nodeid, precinct, l.street as street1, ll.street as street2
                    from {0} as n
                    join {1} as l on l.nodeidfrom::int=nodeid::int
                    join {1} as ll on ll.nodeidto::int=nodeid::int
                    join {2} as p on st_within(n.geom, p.geom) 
                    where l.street !=ll.street
                    )""".format(node_table, lion_table, precinct_table))
    # store locally
    lookup = {}
    # sample 
    print query_to_table(db, "select nodeid, precinct from c_intersection_name limit 10")
    
    data = db.query("select nodeid, precinct from c_intersection_name")
    for row in data[0]:
        node, pct = row
        lookup[node] = pct
    # check if precincts are neighbors (boundary issues)
    data = db.query("""select p1.precinct, p2.precinct
                    from {0} as p1
                    join {0} as p2
                    on st_intersects(p1.geom, p2.geom)
                    """.format(precinct_table))
    neighbors = {}
    bar = progressbar.ProgressBar()
    for row in bar(data[0]):
        p1, p2 = row
        if p1 not in neighbors:
            neighbors[p1] = set()
        neighbors[p1].add(p2)

    to_update = []
    bar = progressbar.ProgressBar()
    for problem in bar(problems):
        rev_lookup = {}
        for node in clusterIntersections[problem][0]:
            if node in lookup.keys():  # known failure for 62493 all streets are "from"
                if lookup[node] not in rev_lookup:
                    rev_lookup[lookup[node]] = set()
                rev_lookup[lookup[node]].add(node)
        if len(rev_lookup) > 1:
            to_update.append(rev_lookup)
    node_master = update_problem_groups(to_update, neighbors, node_master)
    return node_master

params.node_master = check_problem_pcts(db, problems, params.NODE, params.LION, 
                                        params.PRECINCTS, params.clusterIntersections, params.node_master)

In [ ]:
save_dictionaries('OUT')  # 10.07 sec

In [ ]:

nodes = [46298,46393,46398,46402,46404,46406,46408,67047,76634,76638,76662,83615,
         96229,9034370,9034372,9034373,9039520,9039521,9039522,9039525,9039527,9039528]

def get_center(db, nodes):
    # get map center 
    data, col = db.query("""select
                        st_y(st_transform(st_centroid(st_union(geom)),4326)), 
                        st_x(st_transform(st_centroid(st_union(geom)),4326))
                        from node where nodeid in ({})""".format(str(nodes)[1:-1]))
    cent_ln, cent_lt = data[0]
    return cent_ln, cent_lt 

def map_nodes_is_int(db, nodes):
    cent_ln, cent_lt  = get_center(db, nodes) 
    # create map object 
    Map = folium.Map(location=[cent_ln,cent_lt], 
                       tiles='Stamen Terrain',
                       zoom_start=18)
    # get nodes geom
    data, col = db.query("""select nodeid, is_int, 
                         st_y(st_transform(geom, 4326)) as y, 
                         st_x(st_transform(geom, 4326)) as x 
                         from node where nodeid in ({})""".format(str(nodes)[1:-1]))
    # add nodes to map
    for (node, is_int, lon, lat) in data:
        if is_int == -1:
            c = 'red'
        else:
            c = '#3186cc'
        folium.CircleMarker(location=[lon, lat], 
                        radius=10,
                        popup='point {}'.format(node), 
                        color=c,
                        fill_color='#3186cc').add_to(Map)
    #display map 
    return Map
map_nodes_is_int(db, nodes)

In [ ]:

nodes = [41313,41315,41317,41318,41375,41376,41378,81321,82136,82293,82294,82342,82952,
         83000,90111,97255,97256,97650,97651,97653,97715,98042,98230,98238,98240,98407,
         98408,99618,99619,99620,99637,99639,99640,106008,106009,106010,106011,9039786,
         9039810,9039811,9039812,9039813,9039814,9039815,9039826,9039827,9039828,9039829]

def get_center(db, nodes):
    # get map center 
    data, col = db.query("""select
                        st_y(st_transform(st_centroid(st_union(geom)),4326)), 
                        st_x(st_transform(st_centroid(st_union(geom)),4326))
                        from node where nodeid in ({})""".format(str(nodes)[1:-1]))
    cent_ln, cent_lt = data[0]
    return cent_ln, cent_lt 

def map_nodes_is_int(db, nodes):
    cent_ln, cent_lt  = get_center(db, nodes) 
    # create map object 
    Map = folium.Map(location=[cent_ln,cent_lt], 
                       tiles='Stamen Terrain',
                       zoom_start=18)
    # get nodes geom
    data, col = db.query("""select nodeid, is_int, masterid,
                         st_y(st_transform(geom, 4326)) as y, 
                         st_x(st_transform(geom, 4326)) as x 
                         from node where nodeid in ({})""".format(str(nodes)[1:-1]))
    # add nodes to map
    for (node, is_int, master, lon, lat) in data:
        if is_int == -1:
            c = 'red'
        else:
            c = '#3186cc'
        folium.CircleMarker(location=[lon, lat], 
                        radius=10,
                        popup='point {}'.format(master), 
                        color=c,
                        fill_color=c).add_to(Map)
    #display map 
    return Map
map_nodes_is_int(db, nodes)

# Part 5
### Build simple intersections 2 near by nodes

In [ ]:
# run to read saved data back into memory 

params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 se

In [ ]:
def near_nodes(db, node_table, search_distance, node):
    # get nodes within search distance of a given node
    nds = db.query("""select nodeid::int
                    from {0} as nn,
                    (
                            select geom 
                            from {0} 
                            where nodeid = {1}
                    ) as base
                    where st_dwithin(nn.geom, base.geom, {2})
                    and nodeid != {1}
                    and is_int = -1
                    """.format(node_table, node, search_distance))
    return [n[0] for n in nds[0]]

def get_best_master(node_master, master1, master2):
    # find the master with more nodes already associated with it
    mm = {master1: 0, master2: 0}
    for node in node_master.keys():
        if node_master[node] == master1:
            mm[master1] += 1
        elif node_master[node] == master2:
            mm[master2] += 1
        else:
            pass
    if mm[master1] > mm[master2]:
        return master1
    else:
        return master2

@timeDec 
def near_by_simple(db, node_table, node_master, search_distance):
    # get all nodes within seach distance
    bar = progressbar.ProgressBar()
    for node in bar(node_master.keys()):
        n_nodes = near_nodes(db, node_table, search_distance, node)
        for n in n_nodes:
            # go through those nodes
            new_master = get_best_master(node_master, node_master[n], node_master[node])  # put org master 2nd as will default in tie
            # apply the biggest masterid to the node in question
            node_master[node] = new_master
            # apply the biggest masterid to all nodes in question
            node_master[n] = new_master
    return node_master
            
params.node_master = near_by_simple(db, params.NODE, params.node_master, 75)  # 1619.62 sec #26 minutes

In [ ]:
save_dictionaries('OUT')  # 10.07 sec

# Part 6
### Build simple intersections from small triangles 
irregrualr intersections 

In [ ]:
# run to read saved data back into memory 

params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 se

In [ ]:
def closest_n(altGraph, nodeIsIntersection, loc, node, n):
    dl = []
    x, y = loc[node]  # starting position
    for l in altGraph[node]:
        # get coordinates of other end of each seg at start node
        if nodeIsIntersection[l[0]]:
            x1, y1 = loc[l[0]]
            d = distance(x, y, x1, y1)
            # add distance and node to list
            dl.append([d, l[0]])
    z = sorted(dl, key=itemgetter(0))
    # return closest N nodes with their distances
    return z[:n]

@timeDec
def triangle(folder, node_master, altGraph, nodeIsIntersection, triangle_dist=150, n=2):
    # load in the location dict
    loc = cpickle.load(open(os.path.join(folder, "loc.p"), "rb"))
    bar = progressbar.ProgressBar()
    for node in bar(node_master.keys()):
        # for each node get the closest 2
        near = closest_n(altGraph, nodeIsIntersection, loc, node, n)
        # if the furthest near node is within tolerance
        if len(near) > 1:
            if near[-1][0] <= triangle_dist:
                # check can get to each in 1 hop
                # set of 2 closest is subset of nodes that are 1 hop in graph
                # clean up for double segs (vertical topography?)
                sn = set(n[1] for n in near)
                if len(sn) > 1:
                    a, b = set(n[1] for n in near)
                    if a in set(n[0] for n in altGraph[b]):
                        # if set(n[1] for n in near).issubset(set(n[0] for n in altGraph[node])):
                        # should be 1 master
                        for othernode in set(n[1] for n in near):
                            # go through those nodes
                            new_master = get_best_master(node_master,
                                node_master[othernode], node_master[node])  # put org master 2nd as will default in tie
                            # apply the biggest masterid to the node in question
                            node_master[node] = new_master
                            node_master[othernode] = new_master
    return node_master

params.node_master = triangle(params.FOLDER, params.node_master, params.altGraph, params.nodeIsIntersection, 150, 2) 

In [ ]:
save_dictionaries('OUT')  # 10.07 sec

# Part 7
### update database intersections and build master segments


In [ ]:
# run to read saved data back into memory 

params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 se

In [ ]:

@timeDec
def update_db_nodes(db, folder, node_table, node_master):
    # prep dict to write to csv
    x = [[node, node_master[node]] for node in node_master]
    # write out data
    write_csv(os.path.join(folder, "nodeMaster.csv"), x)#, ['nodeid', 'masterid'])
    # create temp_table
    db.query('drop table if exists nodeMaster; CREATE TABLE nodeMaster (nodeid numeric(10,0), masterid numeric(10,0))')
    
    import_table(db, 'nodeMaster', os.path.join(folder, 'nodeMaster.csv'))
#     raw_input(
#         """PG Console \n\n \copy nodeMaster FROM '%s\ nodeMaster.csv' DELIMITERS ',' CSV HEADER;\n\nDone?""" % folder)
#     clear_output()

    db.query("""update %s
                set masterid = nm.masterid
                from nodemaster as nm
                where node.nodeid = nm.nodeid
                """ % node_table)
    db.query('drop table nodemaster')

update_db_nodes(db, params.FOLDER, params.NODE, params.node_master)  # 63.31 sec

In [ ]:
query_to_table(db, "select * from {} where masterid is not null limit 10".format(params.NODE))

In [ ]:

nodes = [46535,46607,46609,46610,46612,46545]

def get_center(db, nodes):
    # get map center 
    data, col = db.query("""select
                        st_y(st_transform(st_centroid(st_union(geom)),4326)), 
                        st_x(st_transform(st_centroid(st_union(geom)),4326))
                        from node where nodeid in ({})""".format(str(nodes)[1:-1]))
    cent_ln, cent_lt = data[0]
    return cent_ln, cent_lt 

    
def map_nodes_is_int(db, nodes):
    cent_ln, cent_lt  = get_center(db, nodes) 
    # create map object 
    Map = folium.Map(location=[cent_ln,cent_lt], 
                       tiles='Stamen Terrain',
                       zoom_start=18)
    # get nodes geom
    data, col = db.query("""select nodeid, is_int, masterid,
                         st_y(st_transform(geom, 4326)) as y, 
                         st_x(st_transform(geom, 4326)) as x 
                         from node where nodeid in ({})""".format(str(nodes)[1:-1]))
    # add nodes to map
    for (node, is_int, master, lon, lat) in data:
        m = params.node_master[46609] # same master
        if master == m:
            c = 'red'
        else:
            c = '#3186cc'
        folium.CircleMarker(location=[lon, lat], 
                        radius=10,
                        popup='point {}'.format(master), 
                        color=c,
                        fill_color=c).add_to(Map)
    #display map 
    return Map
map_nodes_is_int(db, nodes)


In [ ]:
@timeDec
def simple_from_to_masters(db, lion, node):
    print 'Updating Master From/To segments'
    # update the from nodes for simple streets
    # update the to nodes for simple streets

    # make the min master as from and the max master as to
    db.query("""drop table if exists tempFrom; create table tempFrom as (
                    select p1.mft, mn, mx from (
                    select mft, min(masterid) as mn from (
                    select mft, masterid 
                    from {0} as l
                    join {1} as n
                    on nodeidfrom::int = nodeid
                    where mft is not null and masterid is not null
                union
                    select mft, masterid 
                    from {0} as l
                    join {1} as n
                    on nodeidto::int = nodeid
                    where mft is not null and masterid is not null
                    ) as mn group by mft
                    ) as p1
                    join (

                    select mft, max(masterid) as mx from (
                    select mft, masterid 
                    from {0} as l
                    join {1} as n
                    on nodeidfrom::int = nodeid
                    where mft is not null and masterid is not null
                union
                    select mft, masterid 
                    from {0} as l
                    join {1} as n
                    on nodeidto::int = nodeid
                    where mft is not null and masterid is not null
                    ) as mx group by mft) as p2
                    on p1.mft = p2.mft
                    );
                    """.format(lion, node))
    # update lion with mfrom and to values
    db.query("""update %s as l
                    set masteridfrom = mn, masteridto= mx
                    from tempFrom
                    where l.mft = tempFrom.mft 
                    """ % lion)
    db.query("drop table tempFrom ")

simple_from_to_masters(db, params.LION, params.NODE)  # 111.35 sec

query_to_table(db, "select * from {} where street = 'QUEENS BOULEVARD' limit 10".format(params.LION))

In [ ]:

@timeDec
def get_mfts(db, lion):
    # go through list fo mfts
    data = db.query(
        """select mft from {0} where exclude = 0
             group by mft
        """.format(lion))
    # and (masteridfrom is null or masteridto is null)
    local_mfts = [i[0] for i in data[0]]
    return local_mfts


l_mfts = get_mfts(db, params.LION)  # 0.56 sec

In [ ]:
@timeDec
def make_c_master_seg_table(db, node_table, srid):
    print 'Making new table'

    db.query("drop table if exists c_master_segs")
    db.query("""CREATE TABLE c_master_segs ( 
              mft    INTEGER,
              street    varchar(100),
              segmentid    varchar(10),
              geom    geometry(LINESTRING, %i),
              masteridfrom    INTEGER, 
              masteridto    INTEGER,
              seg_cnt INTEGER
              );""" % srid)

    db.query("drop table if exists c_master_node")
    db.query("""create table c_master_node as (
                select distinct masterid,
                        st_centroid(st_union(geom)) as geom,
                        st_x(st_centroid(st_union(geom))) as centerX,
                        st_y(st_centroid(st_union(geom))) as centerY,
                        count(*)::int as nodes
                from %s 
                where masterid !=0
                group by  masterid
                )""" % node_table)

make_c_master_seg_table(db, params.NODE, params.SRID)  # 1.58 sec
query_to_table(db, "select * from c_master_node limit 10")

In [ ]:
@timeDec
def mft_round2(db, lion):
    print 'Updating mft (round 2)'
    # make temp table for segs with same from and to masters and update lion with results
    db.query("""drop table if exists _temp_mft_update_;

                create table _temp_mft_update_ as (
                select min(mft) as mft,masteridfrom, masteridto
                from {0}
                where mft is not null and masteridfrom is not null and masteridto is not null
                group by masteridfrom, masteridto);

                update {0} as l
                set mft = t.mft
                from _temp_mft_update_ as t
                where l.masteridfrom = t.masteridfrom
                and l.masteridto = t.masteridto;

                drop table if exists _temp_mft_update_;
                """.format(lion))

# TODO seems to be missing master segs for park ave BK, but the data is valid in lion and lion tamed
mft_round2(db, params.LION)  # 63.77 sec

query_to_table(db, "select * from {} where street = 'QUEENS BOULEVARD' and rb_layer in ('B', 'G') limit 10".format(params.LION))

In [ ]:
save_dictionaries('OUT')  # 10.07 sec

# Part 8
### update database with final data


In [ ]:
# run to read saved data back into memory 

params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 se

In [ ]:
def get_simple_segs(db, node_table):
    data = db.query("""select mft, st_astext(ST_LineMerge(geom)), simple.masterid,simple2.masterid
                from lion
                ,(
                    select n.nodeid::int, n.masterid
                    from {0} as n
                    join c_master_node
                    on n.masterid = c_master_node.masterid
                    where n.geom =c_master_node.geom
                ) as simple
                ,(
                    select n.nodeid::int, n.masterid
                    from {0} as n
                    join c_master_node
                    on n.masterid = c_master_node.masterid
                    where n.geom =c_master_node.geom
                ) as simple2
                where nodeidfrom::int = simple.nodeid and nodeidto::int = simple2.nodeid
                """.format(node_table))
    simple_mfts = set()
    line = []
    bar = progressbar.ProgressBar()
    for row in bar(data[0]):
        mft, mft_line, mfrom, mto = row
        simple_mfts.add(mft)
        line.append([mft, mft_line, mfrom, mto])
    return simple_mfts, line


def get_compex_segs(db, lion):
    # create table with a count of verticies where are more than 2 so they can be remapped
    db.query("""drop table if exists complexSegs; create table complexSegs as (
                    select mft, max(pos)
                    from (
                            select mft, 
                            (ST_DumpPoints(ST_LineMerge(ST_Union(geom)))).path[1] as pos
                            from %s
                            group by  mft
                                    )as mx
                    where mft is not null
                    group by mft
                    having max(pos) > 2 ) """ % lion)
    # store the mfts that need to be remapped
    data = db.query("select mft from complexSegs")
    to_do = set()
    for mft in data[0]:
        to_do.add(mft[0])
    return to_do


def coord_from_master_qry(db, coordFromMaster):
    
    data = db.query("select  masterid, st_x(geom), st_y(geom) from c_master_node")
    for row in data[0]:
        master, x, y = row
        coordFromMaster[master] = [x, y]
    return coordFromMaster


def get_geom(db, lion):
    d = {}
    data = db.query("""select mft, ST_asText(ST_LineMerge(ST_Union(geom))) as g
                ,masteridfrom, masteridto
                from %s
                where mft is not null and masteridfrom is not null and masteridto is not null
                group by mft, masteridfrom, masteridto;
                """ % lion)
    for output in data[0]:
        mft, geom, mfrom, mto = output
        d[mft] = [geom, mfrom, mto]
    return d

def simple_new_ends(coordFromMaster, p1, p2):
    return "LINESTRING (%s %s, %s %s)" % (str(coordFromMaster[p1][0]), str(coordFromMaster[p1][1]),
                                          str(coordFromMaster[p2][0]), str(coordFromMaster[p2][1]))


def simplify_coord(wkt):
    x = str(wkt.split("(")[1]).split(")")[0]
    if "," not in x:
        x = x.replace("[", '').replace(" ", ",")
    return [float(i) for i in x.split(",")]

def clean(pt):
    if type(pt) != list:
        return [simplify_coord(pt)[0], simplify_coord(pt)[1]]
    else:
        return pt

def coord_to_wkt(coord):
    coord = str(coord)[1:-1].replace(",", "")
    return "POINT(" + coord + ")"

def get_new_ends(db, geom, lion, coordFromMaster):
    # store midles
    d = {}  # {mft: [1st vertex, 2nd vertex...]}
    # get geom without first or last vetex
    data = db.query("""select t.mft, pos, st_astext(geom )
                    from (
                        select mft,
                        (ST_DumpPoints(ST_LineMerge(ST_Union(geom)))).path[1] as pos
                        , (ST_DumpPoints(ST_LineMerge(ST_Union(geom)))).geom as geom
                        from %s
                        where masteridfrom is not null and masteridto is not null 
                        group by mft
                    ) as t
                    join complexSegs as cs 
                    on t.mft=cs.mft
                    where pos > 1 and pos < cs.max""" % lion)
    for row in data[0]:
        mft, pos, coord = row
        if mft not in d.keys():
            d[mft] = [coord]
        else:
            d[mft].append(coord)
    # find closest end point to from or to
    bar = progressbar.ProgressBar()
    for mft in bar(d.keys()):
        start = clean(d[mft][0])
        end = clean(d[mft][-1])
        frm = clean(coordFromMaster[geom[mft][1]])
        to = clean(coordFromMaster[geom[mft][2]])
        # check if only 1 vertex left and rebuild collection
        if start == end:
            d[mft] = [coord_to_wkt(frm), coord_to_wkt(start), coord_to_wkt(to)]
        else:
            # check if start is closer to from or to
            dist = distance(start[0], start[1], frm[0], frm[1])  # SF
            if distance(start[0], start[1], to[0], to[1]) < dist:  # ST
                dist = distance(start[0], start[1], to[0], to[1])
                # start is closer to to, but need to check that start/to is closer that end/to
                if distance(end[0], end[1], to[0], to[1]) < dist:  # ET
                    d[mft] = [coord_to_wkt(frm)] + d[mft] + [coord_to_wkt(to)]
                else:
                    d[mft] = [coord_to_wkt(to)] + d[mft] + [coord_to_wkt(frm)]  # ST
            else:  # SF
                # start is closer to frm, but need to check that start/frm is closer that start/frm
                if distance(end[0], end[1], frm[0], frm[1]) < dist:  # EF
                    d[mft] = [coord_to_wkt(to)] + d[mft] + [coord_to_wkt(frm)]  # ST
                else:
                    d[mft] = [coord_to_wkt(frm)] + d[mft] + [coord_to_wkt(to)]  # SF
        # clean up and make line
        # print 'Finishing mft %i' %(mft)
        d[mft] = line_from_pts(d[mft]).replace("'", "")
    return d

def line_from_pts(wkt_list):
    out = 'LINESTRING('

    def simple(pt):
        return [float(j) for j in pt.split("(")[1].split(")")[0].split(" ")]

    for i in wkt_list:
        out = out + str(simple(i)[0]) + " " + str(simple(i)[1]) + ", "
    return out[:-2] + ")"

In [ ]:

@timeDec
def remap(db, folder, node_table, lion, srid, coordFromMaster):
    # change insert 1 row at a time to csv io model
    simple_mfts, list_c_master_segs = get_simple_segs(db, node_table)
    complex_mfts = get_compex_segs(db, lion)
    # get coordinates for new end points
    params.coordFromMaster = coord_from_master_qry(db, params.coordFromMaster)
    # get basic info for corridors
    geom = get_geom(db, lion)
    counter = 0
    print 'found %i segments that are simple and do not need remapping' % (len(list_c_master_segs))
    todo = []
    bar = progressbar.ProgressBar()
    for mft in bar(geom.keys()):
        if mft in simple_mfts:
            # no change needed
            pass
        elif mft in complex_mfts:
            # check if simple or complex change
            pass
        else:
            todo.append(mft)
    for mft in todo:
        geo, mfrom, mto = geom[mft]
        if 'MULTILINESTRING' in geo:
            pass
        elif mft and mfrom and mto:
            # simple straight  line, just use master from/to as new coordinates
            list_c_master_segs.append([mft, simple_new_ends(coordFromMaster, mfrom, mto), mfrom, mto])

    complex_geom = get_new_ends(db, geom, lion, coordFromMaster)
    bar = progressbar.ProgressBar()
    for mft in bar(complex_geom.keys()):
        new_geom, mfrom, mto = complex_geom[mft], geom[mft][1], geom[mft][2]
        # change insert 1 row at a time to csv io model
        if 'MULTILINESTRING' in new_geom:
            pass
        else:
            list_c_master_segs.append([mft, new_geom, mfrom, mto])
            counter += 1
    db.query("drop table complexSegs")

    print 'done re-mapping %i sgements' % counter
    write_csv(os.path.join(folder, "tempMaster.csv"), list_c_master_segs, None, ';')#, ['mft', 'mftLine', 'mfrom', 'mto'])
    db.query("drop table if exists tempMaster; CREATE TABLE tempMaster (mft int, mftLine text, mfrom int, mto int)")
    import_table(db, 'tempMaster', os.path.join(folder, 'tempMaster.csv'),';')
#      raw_input(
#          """PG Console \n\n \copy tempMaster FROM '%s\\tempMaster.csv' DELIMITERS ',' CSV HEADER;\n\nDone?""" % folder)
#     clear_output()
    db.query("""insert into c_master_segs (mft, geom, masteridfrom, masteridto)
                ((select mft, st_geomfromtext(mftLine, %i), mfrom, mto from tempMaster
                where mft is not null group by mft, mftline, mfrom, mto))""" % srid)
    db.query("drop table tempMaster")

remap(db, params.FOLDER, params.NODE, params.LION, params.SRID, params.coordFromMaster)  # 122.62 sec

query_to_table(db, "select * from c_master_segs limit 10")

In [ ]:
def update_c_master_segs(db, lion):
    db.query("""update c_master_segs as ms
                    set segmentid = q.segmentid,street = q.street
                    from 
                    (
                    select street, lion.segmentid , lion.mft
                    from {0} 
                    , (
                            select mft, segmentid, max(st_length(geom)) as ml
                            from {0}
                            group by mft, segmentid, st_length(geom)
                    ) as mln
                    where {0}.segmentid = mln.segmentid
                    group by street, {0}.segmentid , {0}.mft
                    ) as q
                    where ms.mft=q.mft
                """.format(lion))
    db.query("""update c_master_segs as ms
                    set seg_cnt = cnt
                    from 
                    (
                            select mft, count(*) as cnt from %s group by mft
                    ) as q
                    where ms.mft=q.mft;
                """ % lion)

update_c_master_segs(db, params.LION)


In [ ]:
def stringify(int_to_sting, length):
    int_to_sting = str(int_to_sting)
    if len(int_to_sting) == length:
        return int_to_sting
    else:
        return stringify("0" + int_to_sting, length)
    
@timeDec
def update_roadbeds(db, folder, lion, tbl_rpl):
    data = db.query("select segmentid, mft from %s where mft is not null" % lion)
    seg_mft = {}
    to_test = set()
    for i in data[0]:
        seg, mft = i
        seg_mft[seg] = mft

    data = db.query("""select segmentid, other from (
                    select segmentid, segmentidg as other 
                    from {0} as l
                    join {1} on l.segmentid::int = segmentidr 
                    union
                    select segmentid, segmentidr as other 
                    from {0} as l
                    join {1} on l.segmentid::int = segmentidg
                    ) as base group by segmentid, other
                     """.format(lion, tbl_rpl))
    bar = progressbar.ProgressBar()
    for row in bar(data[0]):
        seg1, seg2 = row[0], stringify(row[1], 7)
        if seg1 in seg_mft.keys():
            # seg2 might not be in the dict
            if seg2 not in seg_mft.keys():
                # add rb to dict with centerline mft
                seg_mft[seg2] = seg_mft[seg1]
        elif seg2 in seg_mft.keys():
            # seg2 might not be in the dict
            if seg1 not in seg_mft.keys():
                # add rb to dict with centerline mft
                seg_mft[seg1] = seg_mft[seg2]
        else:
            to_test.add(seg1)
            to_test.add(seg2)

    print 'Still missing %i segments\n' % (len(to_test))
    out = [[key, seg_mft[key]] for key in seg_mft]

    write_csv(os.path.join(folder, "tempMaster.csv"), out)#, ['Segmentid', 'MFT'])

    db.query("drop table if exists tempMaster; CREATE TABLE tempMaster (Segmentid int, MFT int)")

    import_table(db, 'tempMaster', os.path.join(folder, 'tempMaster.csv'))
#     raw_input(
#         """PG Console \n\n \copy tempMaster FROM '%s\\tempMaster.csv' DELIMITERS ',' CSV HEADER;\n\nDone?""" % folder)
#     clear_output()
    db.query("""update %s as l
                    set mft = t.mft
                    from tempMaster as t
                    where l.segmentid::int = t.segmentid
                    and l.mft is null""" % lion)

    db.query("drop table tempMaster")

    db.query("""update %s as l
                    set masteridfrom = m.masteridfrom, masteridto = m.masteridto
                    from c_master_segs as m
                    where l.mft = m.mft""" % lion)

update_roadbeds(db, params.FOLDER, params.LION, params.RPL)  # 1302.61 sec
query_to_table(db, "select * from c_master_segs limit 10")

In [ ]:

@timeDec
def update_roadbed_nodes(db, node_table, tbl_rpl):
    db.query("""create table m as (
                select n.nodeid, nn.masterid
                    from {0} as n
                    join {1} as t
                        on n.nodeid=t.r_tond
                    join {0} as nn
                        on g_tond = nn.nodeid
                    where n.masterid is null and nn.masterid is not null
                union
                    select n.nodeid, nn.masterid
                    from {0} as n
                    join {1} as t
                        on n.nodeid=t.g_tond
                    join {0} as nn
                        on r_tond = nn.nodeid
                    where n.masterid is null and nn.masterid is not null
                union
                    select n.nodeid, nn.masterid
                    from {0} as n
                    join {1} as t
                        on n.nodeid=t.r_frnd
                    join {0} as nn
                        on g_frnd = nn.nodeid
                    where n.masterid is null and nn.masterid is not null
                union
                    select n.nodeid, nn.masterid
                    from {0} as n
                    join {1} as t
                        on n.nodeid=t.g_frnd
                    join {0} as nn
                        on r_frnd = nn.nodeid
                    where n.masterid is null and nn.masterid is not null)
                """.format(node_table, tbl_rpl))

    for i in range(1, 5):
        db.query("""update %s as n
                    set masterid = m.masterid
                    from m
                    where n.nodeid=m.nodeid and n.masterid is null""" % node_table)
    db.query("drop table m")
    db.query("""update {0} as n
                    set masterid  = nn.masterid
                    from (
                        select distinct r,g from (
                        select r_frnd as r, g_frnd as g
                        from {1}
                        union
                        select r_tond as r, g_tond as g
                        from {1}
                        ) as passthrough
                    )as nl, {0} as nn
                    where n.nodeid::int = r::int
                    and  g::int = nn.nodeid::int
                    and n.masterid is null and nn.masterid is not null
                """.format(node_table, tbl_rpl))
    db.query("""update {0} as n
                    set masterid  = nn.masterid
                    from (
                        select distinct r,g from (
                        select r_frnd as r, g_frnd as g
                        from {1}
                        union
                        select r_tond as r, g_tond as g
                        from {1}
                        ) as passthrough
                    )as nl, {0} as nn
                    where n.nodeid::int = g::int
                    and  r::int = nn.nodeid::int
                    and n.masterid is null and nn.masterid is not null
                """.format(node_table, tbl_rpl))
    
update_roadbed_nodes(db, params.NODE, params.RPL)
query_to_table(db, "select * from {0} limit 10".format(params.NODE))

## Regenerate MFTs to be longest segment and MasterIDs for most central NodeID to allow for continuity

In [ ]:

def make_new_mft_lookup(db, lion):
    db.query("drop table if exists temp_new_mfts;")
    db.query("""create table temp_new_mfts as 
                select min(segmentid) as seg, l.mft from {0} l join (
                    select mft, max(st_length(geom)) as mxl from {0} where mft is not null group by mft
                    ) as ml on l.mft = ml.mft and st_length(l.geom) = ml.mxl
                group by l.mft;
            """.format(lion))
        
def make_new_masternode_lookup(db, node):
    db.query("drop table if exists temp_new_masters;")
    db.query("""create table temp_new_masters as 
                select d.* 
                from (
                select n.masterid, min(st_distance(n.geom, c.geom)) as mdist
                from {0} n join c_master_node as c on n.masterid = c.masterid
                group by n.masterid
                ) md join (
                select nodeid, n.masterid, st_distance(n.geom, c.geom) as dist
                from {0} n join c_master_node as c on n.masterid = c.masterid
                ) d on md.masterid = d.masterid and mdist = dist
            """.format(node))

def update_tables_masters(db, lion, node):
    mft_tables = ['c_master_segs', lion]
    master_n_tables = [node, 'c_master_node']
    for t in mft_tables:
        db.query("""update {} t set mft = n.seg::int 
                    from temp_new_mfts n where t.mft = n.mft
                """.format(t))
        db.query("""update {} t set masteridfrom = n.nodeid::int
                    from temp_new_masters n where t.masteridfrom = n.masterid
                    """.format(t))
        db.query("""update {} t set masteridto = n.nodeid::int
                    from temp_new_masters n where t.masteridto = n.masterid
                    """.format(t))
    for t in master_n_tables:
        db.query("""update {} t set masterid = n.nodeid::int 
                    from temp_new_masters n where t.masterid = n.masterid
                """.format(t))

make_new_mft_lookup(db, params.LION)
make_new_masternode_lookup(db, params.NODE)
update_tables_masters(db,  params.LION, params.NODE)

In [ ]:
print db.query("select * from temp_new_mfts where mft = 109072")
print db.query("select * from temp_new_masters where masterid = 9683")

In [ ]:

def make_centerline_clean(db, lion, node_table, version):
    db.query("""drop table if exists C_Lion_{0}; create table C_Lion_{0} as (
                    select distinct *
                    from {1}
                    where exclude = 0 and mft is not null
                    )""".format(version, lion))

    db.query("""drop table if exists C_Lion_Nodes_{0}; create table C_Lion_Nodes_{0} as (
                    select distinct *
                    from {1}
                    where is_int =-1
                    )""".format(version, node_table))
    print '\nDONE!\n'

make_centerline_clean(db, params.LION, params.NODE, params.VERSION)

query_to_table(db, "select * from  C_Lion_{0} limit 10".format(params.VERSION))

In [ ]:
@timeDec
def add_boroughs_to_intersections(db, version, boroughs):
    nodes = 'C_Lion_nodes_' + version
    print 'Updating boroughs for nodes\n'
    db.query("""alter table {0} drop column if exists lboro;
                    alter table {0} drop column if exists rboro;
                    alter table {0} add column lboro int;
                    alter table {0} add column rboro int;
                    """.format(nodes))

    db.query("""update %s as n
                    set lboro = borocode
                    from %s as b
                    where st_dwithin(n.geom, b.geom, 5)
                    and lboro is null;""" % (nodes, boroughs))
    db.query("""update %s as n
                    set rboro = borocode
                    from %s as b
                    where st_dwithin(n.geom, b.geom, 5)
                    and rboro is null;""" % (nodes, boroughs))

add_boroughs_to_intersections(db, params.VERSION, params.BOROUGHS)  # 400.35 sec
query_to_table(db, "select * from  C_Lion_nodes_{0} limit 10".format(params.VERSION))

In [ ]:
def get_nodes(db, node_table):
    data = db.query(
        "select count(*) from (select distinct masterid from %s where masterid is not null) as t" % node_table)
    return data[0][0]

@timeDec
def make_lookup(db, node_table, lion_table, ver):
    db.query("drop table if exists c_lion_node_lookup")

    db.query('''create table c_lion_node_lookup as (
                select nodeid, masterid, mft, segmentid
                from %s as l
                join %s as n
                 on l.nodeidfrom::int = n.nodeid
                union
                select nodeid, masterid, mft, segmentid
                from %s as l
                join %s as n
                 on l.nodeidto::int = n.nodeid
                ) ''' % (lion_table, node_table, lion_table, node_table))
    # index lookup
    version = 'C_Lion_nodes_'+ver
    index_list = ["drop index if exists nd_IDX; ", "drop index if exists master_IDX;", "drop index if exists mft_IDX;",
                  "drop index if exists seg_IDX;", "drop index if exists lt_node_IDX",
                  "CREATE INDEX nd_IDX ON c_lion_node_lookup (nodeid);",
                  "CREATE INDEX master_IDX ON c_lion_node_lookup (masterid);",
                  "CREATE INDEX mft_IDX ON c_lion_node_lookup (mft);",
                  "CREATE INDEX seg_IDX ON c_lion_node_lookup (segmentid);",
                  "CREATE INDEX lt_node_IDX ON %s (nodeid);" % version]
    for idx in index_list:
        db.query(idx)

make_lookup(db, params.NODE, params.LION, params.VERSION)  # 122.01 sec

In [ ]:
# clean up directory 

for f in os.listdir(params.FOLDER):
    if f.endswith(('.csv','.p')):
        print 'Deleting {}...'.format(f)
        os.remove(f)
        
        
# grant permissions
tables = ['node_stnameft', 'lion', 'node', 'c_master_node',
              'c_master_segs', 'C_Lion_%s' % params.VERSION, 'C_Lion_Nodes_%s' % params.VERSION]
for t in tables:
    db.query('grant all on {} to public'.format(t))

In [ ]:
def get_mileage(db, lion):
    miles = {}
    for b in range(1, 6):
        data = db.query("""select sum(st_length(geom))/5280 from (
                        select distinct on (segmentid) geom from {0}
                        where exclude = 0
                            and mft is not null
                            and (lboro = {1} or rboro = {1} )
                        group by segmentid, geom
                    ) as singles""".format(lion, b, b))
        miles[b] = data[0][0][0]
    data = db.query("""select sum(st_length(geom))/5280 from ( 
                    select distinct on (segmentid) geom from {0}
                    where exclude = 0
                        and mft is not null
                    group by segmentid, geom
                ) as singles""".format(lion))
    miles[0] = data[0][0][0]
    return miles

miles = get_mileage(db, params.LION)

In [ ]:
print '-----------------------------------------------------------------------------------'
print 'LION (Version {0}) files:\nBasic ({1}, {2}), \nUgly(c_master_segs, c_master_node), \
\nCenterline(C_Lion{0}, C_Lion_Nodes_{0})'.format(
    params.VERSION, params.LION, params.NODE)
print '-----------------------------------------------------------------------------------'
for i in miles:
    if i > 0:
        print 'Borough %i: %4.2f Miles of Centerline Non-Highway Roadway' % (i, miles[i])
    else:
        print 'Citywide: %4.2f Miles of Centerline Non-Highway Roadway' % (miles[i])
print '%i Intersections' % (get_nodes(db, params.NODE))
print '-----------------------------------------------------------------------------------'

In [ ]:
df = query_to_table(db,"""
                    select lboro as Borough, sum(st_length(geom))/5280 as Miles from (
                        select distinct on (segmentid) lboro, geom from {0}
                        where exclude = 0
                            and mft is not null
                        group by segmentid, geom, lboro
                    ) as singles group by lboro """.format(params.LION))

cols = ['borough', 'miles']
df = pd.DataFrame(df.to_dict(), columns=cols) # convert to pandas dataframe
#print df

labels = df['borough']
sizes = df['miles']


fig1, ax1 = plt.subplots()
ax1.set_title('Miles by Borough\n')
ax1.pie(sizes, labels=labels, autopct='%1.0f%%')#, startangle=70)
ax1.axis('equal')  

plt.show()
